<a href="https://colab.research.google.com/github/Neetagrg/Mini-Project_2_Spring-2025-WEB-DATA-MINING-CUS-635/blob/main/mini_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project 2: Neeta Kumari & Bir Bahadur Gharti Class CUS635

# Install Required Libraries

In [ ]:
!pip install boto3 requests pandas

# AWS Connection

In [ ]:
# Import the boto3 library
import boto3
from botocore.config import Config
from botocore import UNSIGNED

# Team 6 folder
TEAM = "TEAM_6/"

# Define the provided S3 bucket name
BUCKET_NAME = "cus635-spring2025"

# Create an anonymous S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Upload the CSV to S3 Download a File from S3

In [ ]:
import boto3
from botocore.config import Config
from botocore import UNSIGNED

# Team
TEAM = "TEAM_6/"
BUCKET_NAME = "cus635-spring2025"

# Create an anonymous S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Download a file from S3
object_name = "TEAM_6/Financial_Articles.csv"  # Corrected file path
new_name = "downloaded_Finance_Articles.csv"

s3.download_file(BUCKET_NAME, object_name, new_name)
print(f"File '{object_name}' downloaded successfully as '{new_name}'!")

File 'TEAM_6/Financial_Articles.csv' downloaded successfully as 'downloaded_Finance_Articles.csv'!


# Pinecone Setup

In [ ]:
pip install boto3 pinecone-client tqdm


In [ ]:
!pip install pinecone

In [ ]:
from pinecone import Pinecone

# Initialize Pinecone instance with your API key and environment
pc = Pinecone(
    api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6",
    environment="us-east-1"
)

# Index name you're trying to access
index_name = "cus635"

# Check if the index exists
if index_name not in pc.list_indexes().names():
    print(f"Index '{index_name}' does not exist.")
else:
    print(f"Successfully connected to index '{index_name}'.")

    # Access your index
    index = pc.Index(index_name)


Successfully connected to index 'cus635'.


# Create the DocumentProcessor Class

In [ ]:
class DocumentProcessor:
    def __init__(self, team, category):
        self.team = team
        self.category = category

    def fetch_documents(self, bucket, prefix):
        response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        return [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.json')]

    def process_and_upload(self, bucket, file_key):
        obj = s3.get_object(Bucket=bucket, Key=file_key)
        content = json.loads(obj['Body'].read().decode('utf-8'))

        # Assume JSON contains at least 'title', 'text', 'published'
        vector = {
            "id": file_key,  # or generate a unique ID
            "values": None,  # Leave this None to use automatic embedding
            "metadata": {
                "title": content.get('title', 'No Title'),
                "text": content.get('text', ''),
                "published": content.get('published', ''),
                "team": self.team,
                "category": self.category
            }
        }
        index.upsert(vectors=[vector])
        print(f"Uploaded: {file_key}")


Upsert into Pinecone (EMBEDDINGS HANDLED AUTOMATICALLY)


UUID: Each document gets a unique ID (uuid4()).

Embedding: Since your index uses the llama-text-embed-v2 embedding model, embeddings are handled automatically when you pass the text data.

Upsert: The index.upsert(vectors=vectors) uploads your documents to Pinecone.*italicized text*

In [ ]:
from uuid import uuid4
import pandas as pd

# Load your data (example: CSV with a 'content' column)
df = pd.read_csv("your_file.csv")

# Convert your content column into Pinecone-compatible vectors
vectors = [(str(uuid4()), {"text": row["content"]}) for _, row in df.iterrows()]

# Upsert the vectors into Pinecone
index.upsert(vectors=vectors)



FileNotFoundError: [Errno 2] No such file or directory: 'your_file.csv'

# Read and Preprocess CSV

In [ ]:
import pandas as pd

# Load the downloaded CSV
df = pd.read_csv("downloaded_Finance_Articles.csv")

# View basic info
df.head()

# Drop rows with missing context
df.dropna(subset=['context'], inplace=True)

# Clean the text (basic cleanup)
def clean_text(text):
    return text.strip().replace('\n', ' ').replace('\r', '').lower()

# Apply cleaning to 'context' column and create a new column 'clean_content'
df['clean_content'] = df['context'].apply(clean_text)

# Optional: preview the result
df[['heading', 'clean_content']].head()


,heading,clean_content
0,Motor finance compensation ruling ‘goes too fa...,<p>a court of appeal ruling that has left lend...
1,Supreme court judges reject Reeves’ motor fina...,<p>rachel reeves was dealt a fresh blow on mon...
2,Car finance firm Close Brothers slumps to loss...,"<p>close brothers, one of the uk’s biggest pro..."
3,Chief finance officer of under-fire Thames Wat...,<p>thames water’s chief financial officer has ...
4,Car finance scandal: lender sets aside £165m f...,<p>one of the uk’s biggest providers of motor ...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('heading').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('clean_content').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('heading')):
  _plot_series(series, series_name, i)
  fig.legend(title='heading', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['clean_content'].value_counts()
    for x_label, grp in _df_6.groupby('heading')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('heading')
_ = plt.ylabel('clean_content')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['heading'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='index', y='heading', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['clean_content'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='clean_content', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

# Chunk / Batch Articles

To make documents manageable for Pinecone’s index

In [ ]:
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Explode all documents into chunks
chunked_docs = []

for i, row in df.iterrows():
    chunks = chunk_text(row['clean_content'])
    for idx, chunk in enumerate(chunks):
        chunked_docs.append({
            'id': f"{i}-{idx}",
            'text': chunk,
            'metadata': {'source': f"article_{i}", 'category': 'finance'}
        })

print(f"Total Chunks: {len(chunked_docs)}")


Total Chunks: 1000


# Connect to Pinecone

In [ ]:
import pinecone

# Instead of pinecone.init(), create a Pinecone instance:
pinecone = pinecone.Pinecone(
    api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6",
    environment="us-east-1-aws"
)

index_name = "cus635"

# Confirm index exists
if index_name not in pinecone.list_indexes().names(): # Update to access names attribute
    raise ValueError(f"Index {index_name} does not exist.")

index = pinecone.Index(index_name)

# Upsert Data (Upload)

Since embedding is handled automatically, just upsert the documents Load and Chunk the Text

In [ ]:
import pandas as pd
from pinecone import Pinecone

# Load the CSV file
df = pd.read_csv("downloaded_Finance_Articles.csv")

# Connect to Pinecone with the new SDK
api_key = "pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6"
pc = Pinecone(api_key=api_key)
index = pc.Index("cus635")  # Make sure this index already exists

# Prepare documents
documents = []
for i, row in df.iterrows():
    text = row.get("context") or row.get("text") or row.get("article") or row.get("Title")
    if not text:
        continue
    metadata = {"source": f"article_{i}"}
    documents.append({"text": text, "metadata": metadata})


# Upload to Pinecone (Without Manual Embeddings

Pinecone index is pre-configured to embed using llama-text-embed-v2, DO NOT need manually create embeddings

In [ ]:
import pinecone

# Connect to Pinecone
pinecone.init(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6", environment="us-east-1-gcp")

# Connect to your index
index = pinecone.Index("cus635")

# Push the documents
index.ingest(documents=documents)

print("Documents uploaded and embedded automatically!")


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



DocumentProcessor Class for S3 and Pinecone Integration

In [ ]:
import json
from uuid import uuid4

class DocumentProcessor:
    def __init__(self, team, category):
        self.team = team
        self.category = category

    def fetch_documents(self, bucket, prefix):
        response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        return [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.json')]

    def process_and_upload(self, bucket, file_key):
        obj = s3.get_object(Bucket=bucket, Key=file_key)
        content = json.loads(obj['Body'].read().decode('utf-8'))

        vector = {
            "id": file_key,
            "values": None,  # Embedding handled automatically by Pinecone
            "metadata": {
                "title": content.get('title', 'No Title'),
                "text": content.get('text', ''),
                "published": content.get('published', ''),
                "team": self.team,
                "category": self.category
            }
        }
        index.upsert(vectors=[vector])
        print(f"Uploaded: {file_key}")


# Text Preprocessing and Chunking

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("downloaded_Finance_Articles.csv")

# Clean the text data
def clean_text(text):
    return text.strip().replace('\n', ' ').replace('\r', '').lower()

df['clean_content'] = df['context'].apply(clean_text)

# Chunk the cleaned text into smaller pieces for Pinecone indexing
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

chunked_docs = []
for i, row in df.iterrows():
    chunks = chunk_text(row['clean_content'])
    for idx, chunk in enumerate(chunks):
        chunked_docs.append({
            'id': f"{i}-{idx}",
            'text': chunk,
            'metadata': {'source': f"article_{i}", 'category': 'finance'}
        })
print(f"Total Chunks: {len(chunked_docs)}")


Total Chunks: 1000


# Uploading the Chunks to Pinecone

In [ ]:
import uuid

documents = []
for chunk in chunked_docs:
    document = {
        "id": str(uuid.uuid4()),
        "values": None,  # No embedding needed
        "metadata": {
            "text": chunk['text'],
            **chunk['metadata']
        }
    }
    documents.append(document)

# Upload documents to Pinecone
index.upsert(vectors=documents)
print("Documents uploaded successfully!")


ListConversionException: Expected a list or list-like data structure, but got: None